In [1]:
#!conda install -c anaconda seaborn -y

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
tn_part3 = pd.read_csv('../data/tn_data_pctrnk_risk_calc.csv', index_col=[0])
tn_part3

county  pct_children_under_18  pct_people_living_in_rural_areas  pct_people_of_color  pct_people_with_disabilities  pct_senior_citizens  total_housing_units_2019                   county_full  population_est  pct_insured  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  edu_lessHS  edu_HSgrad  edu_assoc  edu_min_bac  employed_est  unemployed_est  tot_w_poverty_lvl_data  tot_poverty_less100  tot_poverty_101_138  tot_poverty_138_399  tot_poverty_400_up  area_miles2  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  total_vulnerability  pct_uninsured    gdp_2018  gdp_per_cap_2018  mag_mean  mag_median  mag_max  depth_mean  depth_median  depth_min  num_earthquakes  pct_prob_100y_median  pct_prob_100y_max  pop_density  pctrnk_pop_density  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  \
0     anderson                   21.9                             34.66                  6.8                          18.6                 17.4                     35143    Anderson County, Tennessee           74862         90.8       2365       1018       2094         377         29       66911        637        2021         14157         60705        6828       17453      15622        12480         30244            1969                   74292                12022                 5901                33240               23129   337.203175         0.2128                         0.6277                        0.6596                           0.6489               0.4043            9.2   4770850.0         63.728594  1.820000       1.725     2.23   15.995000        15.185      12.98              4.0                16.680              23.48   222.008586            0.842105        0.233136           0.852632       0.208677          0.115789         0.166707            0.136842   
1      bedford                   26.8                             55.60                 16.0                          15.7                 12.9                     19531     Bedford County, Tennessee           47184         88.7       3384         79       5682          27         29       36541       2404        1495          7526         39658        5814       12314       7346         4999         19883             940                   46923                 7420                 4105                24435               10963   473.638959         0.3511                         0.6915                        0.9681                           0.5319               0.6809           11.3   1824402.0         38.665692  2.770000       2.770     2.77    6.520000         6.520       6.52              1.0                16.470              19.56    99.620184            0.663158        0.260978           0.673684       0.155688          0.821053         0.105947            0.463158   
2       benton                   20.5                             78.46                  3.9                          24.6                 19.9                      9172      Benton County, Tennessee           15960         90.3        507         34        274          69          0       14835          0         285          4297         11663        2161        5447       2519         1404          5185             443                   15922                 3403                 2114                 7145                3260   394.306039         0.8085                         0.7766                        0.2553                           0.6277               0.7766            9.7    386699.0         24.229261  0.000000       0.000     0.00    0.000000         0.000      25.35              0.0                30.180              32.83    40.476174            0.221053        0.341291           0.042105       0.157832          0.800000         0.087970            0.736842   
3      bledsoe                   21.5                   

In [5]:
# retain only columns of interest for answering our part 3 questions
tn_part3.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 94
Data columns (total 103 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   county                            object 
 1   pct_children_under_18             float64
 2   pct_people_living_in_rural_areas  float64
 3   pct_people_of_color               float64
 4   pct_people_with_disabilities      float64
 5   pct_senior_citizens               float64
 6   total_housing_units_2019          int64  
 7   county_full                       object 
 8   population_est                    int64  
 9   pct_insured                       float64
 10  race_afam                         int64  
 11  race_asam                         int64  
 12  race_latn                         int64  
 13  race_natam                        int64  
 14  race_paci                         int64  
 15  race_white                        int64  
 16  race_othr                         int64  
 17

In [6]:
# subsetting dataset - only retain variables of interest
tn_df = tn_part3[['county', 'county_full', 'population_est', 'pctrnk_pop_est', 'pop_density', 'pctrnk_pop_density', \
                          'pct_people_living_in_rural_areas', 'gdp_2018', 'pctrnk_gdp_2018', 'gdp_per_cap_2018', \
                          'pctrnk_gdp_per_cap_2018', 'total_housing_units_2019', 'pctrnk_housing_units', 'race_afam', \
                          'race_asam', 'race_latn', 'race_natam', 'race_paci', 'race_white', 'race_othr', 'race_multi', \
                          'disability_y', 'disability_n', 'pct_uninsured', 'pctrnk_uninsured', 'pct_edu_HSgrad', \
                          'pctrnk_edu_HSgrad', 'pct_edu_assoc', 'pctrnk_edu_assoc', 'pct_edu_min_bac', \
                          'pctrnk_edu_min_bac', 'pct_poverty_138_399', 'pctrnk_poverty_138_399', 'pct_poverty_400_up', \
                          'pctrnk_poverty_400_up', 'socioeconomic', 'household_comp_and_disability', \
                          'minority_status_and_language', 'housing_type_and_transportation', 'num_earthquakes', \
                          'pctrnk_earthquakes', 'pct_prob_100y_median', 'pctrnk_prob_100y_median', 'pct_prob_100y_max', \
                          'pctrnk_prob_100y_max', 'mag_mean', 'pctrnk_mag_mean', 'mag_median', 'pctrnk_mag_median', \
                          'mag_max', 'pctrnk_mag_max', 'depth_mean', 'pctrnk_depth_mean', 'depth_median', \
                          'pctrnk_depth_median', 'depth_min', 'pctrnk_depth_min', 'hazard_wcs_both', \
                          'pctrnk_hazard_wcs_both', 'exposure_wcs_both', 'pctrnk_exposure_wcs_both', \
                          'vulnerability_wcs_both', 'pctrnk_vul_wcs_both', 'risk_calc_wcs_both', 'hazard_wcs', \
                          'pctrnk_hazard_wcs', 'exposure_wcs', 'pctrnk_exposure_wcs', 'vulnerability_wcs', \
                          'pctrnk_vul_wcs', 'risk_calc_wcs', 'hazard_norm_both', 'pctrnk_hazard_norm_both', \
                          'exposure_norm_both', 'pctrnk_exposure_norm_both', 'vulnerability_norm_both', \
                          'pctrnk_vul_norm_both', 'risk_calc_norm_both', 'hazard_norm', 'pctrnk_hazard_norm', \
                          'exposure_norm', 'pctrnk_exposure_norm', 'vulnerability_norm', 'pctrnk_vul_norm', 'risk_calc_norm']]

In [14]:
# sort values based on population percent rank
tn_df.sort_values('pctrnk_pop_est', ascending=False).head()

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas    gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
78      shelby      Shelby County, Tennessee          924453        1.000000  1210.691352            0.989474                              2.76  65031814.0         0.021053         70.346263                 0.042105                    407273              1.000000     493772      23866      58081        1576        251      334033      25045       15966        116075        808378           12.1          0.873684        0.173417           0.957895       0.186995          0.389474         0.197581            0.073684             0.392759                0.978947            0.321626               0.157895         0.3617                         0.5638                        0.9894                           0.9468              4.0            0.668421                36.370                 0.905263              41.26              0.905263  2.407500         0.926316        2.35           0.857895     3.03        0.757895    6.192500           0.526316         5.535             0.536842   
18    davidson    Davidson County, Tennessee          677542        0.989474  1345.692089            1.000000                              3.41  71674551.0         0.010526        105.786137                 0.010526                    325454              0.989474     184653      23971      69163        1629        430      381405      16979       17376         78584        598958           12.6          0.926316        0.150305           0.989474       0.169494          0.642105         0.266837            0.021053             0.414120                0.957895            0.351062               0.094737         0.0745                         0.0638                        1.0000                           0.9787              0.0            0.152632                17.710                 0.473684              22.92              0.473684  0.000000         0.157895        0.00           0.157895     0.00        0.157895    0.000000           0.852632         0.000             0.852632   
46        knox        Knox County, Tennessee          452543        0.978947   890.252310            0.978947                             10.92  27307967.0         0.031579         60.343364                 0.094737                    210085              0.978947      39749       9983      18731        1336        174      373681       5138        9271         56682        395861            8.4          0.110526        0.167840           0.978947       0.184378          0.410526         0.240147            0.031579             0.407017                0.968421            0.370562               0.052632         0.0532                         0.0000                        0.8723                           0.5213             59.0            0.926316                22.145                 0.684211              29.02              0.694737  2.096780         0.663158        2.02           0.621053     3.70        0.905263   13.866102           0.189474        14.790             0.157895   
32    hamilton    Hamilton C

#### Observations:  
  
* Memphis (Shelby County), Nashville (Davidson), Knoxville (Knox), and Chattanooga (Hamilton) are the county seats of the four most populous counties in TN. Taking population totals and density into account (population density only in parentheses), their worst case scenario risk scores in the event of an earthquake are 1.0 (0.88), 0.47 (0.19), 0.80 (0.71), and 0.82 (0.68), respectively.  
  
  
* Although these major cities are important, focusing resources on them prioritizes avoidance of potential economic losses (these 4 have the highest GDPs) over actual risk of disaster in the event of an earthquake. For example, USGS researchers estimate between a 17.7% and 22.9% chance of experiencing a potentially ground damaging earthquake (magnitude of VI or higher) in Davidson County in 100 years. This is arguably low compared to places like  Shelby County (probabilities between 36.4% and 41.3%), for example.  
    
    
* The top 5 counties with the highest worst case scenario risk scores are Union (1.0), Grainger (0.99), Campbell (0.98), Monroe (0.97), and Lawrence (0.96) - accounting for population density only  
* The top 5 counties with the highest worst case scenario risk scores are Shelby (1.0), Bradley(0.99), Monroe (0.98), Obion (0.97), Lauderdale (0.96) - accounting for population density and total population  
    
    
* Top 5 counties with highest mid-range risk scores are Lawrence (1), Shelby (0.99), Grainger (0.98), Union (0.97), and Meigs (0.96), - accounting for population density only  
* Top 5 counties with highest mid-range risk scores are Shelby (1), Lawrence (0.99), Warren (0.98), Lauderdale (0.97), and Hamblin (0.96) - accounting for population density and total population  

In [8]:
# sort values based on worst case scenario risk (pop_density only)
tn_df.sort_values('risk_calc_wcs', ascending=False).head(10)

county                  county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas   gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
86      union      Union County, Tennessee           19119        0.315789    85.515191            0.631579                            100.00   352576.0         0.800000         18.441132                 0.936842                      9573              0.326316         58         33        299           6         44       18460         25         248          3610         15509           11.9          0.842105        0.316335           0.136842       0.150165          0.894737         0.058267            0.989474             0.500526                0.284211            0.177684               0.884211         0.9255                         0.2660                        0.0851                           0.8723             11.0            0.752632                16.440                 0.368421              19.59              0.252632  2.283636         0.810526       2.310           0.831579     3.37        0.873684   18.329091           0.052632        18.670             0.063158   
28   grainger   Grainger County, Tennessee           22870        0.378947    81.507868            0.600000                            100.00   426775.0         0.757895         18.660909                 0.915789                     11214              0.400000         94         75        742          88          0       21622          1         249          5446         17424           10.6          0.621053        0.312462           0.147368       0.172715          0.610526         0.072759            0.905263             0.491739                0.389474            0.218784               0.642105         0.6702                         0.4894                        0.4468                           0.3191             29.0            0.863158                16.685                 0.410526              21.06              0.352632  2.135172         0.694737       2.110           0.715789     4.10        0.942105   11.993571           0.294737        11.360             0.273684   
6    campbell   Campbell County, Tennessee           39122        0.589474    81.476577            0.589474                             55.02   917213.0         0.536842         23.444941                 0.800000                     21394              0.652632         85        155        473         161          0       37831        129         421          9813         29309           11.3          0.763158        0.290041           0.442105       0.154338          0.842105         0.084760            0.789474             0.488573                0.410526            0.201206               0.757895         0.8404                         0.9255                        0.0213                           0.6596             14.0            0.778947                14.455                 0.200000              16.57              0.105263  2.373571         0.884211       2.400           0.894737     3.80        0.915789   20.656429           0.031579        19.875             0.042105   
61     monroe     Monroe County, Tennes

In [24]:
tn_df.sort_values('pctrnk_vul_wcs', ascending=False).head()

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas   gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
48  lauderdale  Lauderdale County, Tennessee           23529        0.410526    49.853072            0.305263                             58.66   760976.0         0.610526         32.342046                 0.505263                     11323              0.410526       7416         60        570         198          0       14711        183         582          5272         18257           10.3          0.578947        0.282120           0.494737       0.165498          0.673684         0.063029            0.968421             0.482881                0.505263            0.169786               0.957895         0.9362                         0.9362                        0.8085                           0.9894             40.0            0.915789                48.255                 0.968421              54.39              0.968421  2.133250         0.684211       2.025           0.631579     4.00        0.926316      8.3850           0.389474         7.850             0.378947   
30      grundy      Grundy County, Tennessee           13152        0.157895    36.488247            0.168421                            100.00   238649.0         0.884211         18.145453                 0.947368                      6480              0.168421         66         25        164         118          6       10994         13        1770          3259          9893           17.0          1.000000        0.288397           0.452632       0.147962          0.936842         0.082117            0.821053             0.471777                0.642105            0.175286               0.915789         0.9894                         0.9894                        0.5000                           0.8936              4.0            0.668421                17.570                 0.463158              22.60              0.452632  1.747500         0.410526       1.700           0.394737     2.29        0.421053     14.2775           0.168421        13.780             0.168421   
88      warren      Warren County, Tennessee           39922        0.621053    92.268233            0.642105                             61.38  1421164.0         0.357895         35.598517                 0.421053                     18053              0.610526        540        241       3601         141        119       34267        873        1212          8214         31708           12.4          0.905263        0.290366           0.431579       0.148139          0.915789         0.094660            0.642105             0.455339                0.810526            0.218866               0.631579         0.7660                         0.8830                        0.8936                           0.5851              0.0            0.152632                19.190                 0.557895              20.84              0.336842  0.000000         0.157895       0.000           0.157895     0.00        0.157895      0.0000           0.852632         0.000             0.852632   
49    lawrence    Lawrence Count

In [9]:
# sort values based on worst case scenario risk (pop_density + population)
tn_df.sort_values('risk_calc_wcs_both', ascending=False).head(10)

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas    gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
78      shelby      Shelby County, Tennessee          924453        1.000000  1210.691352            0.989474                              2.76  65031814.0         0.021053         70.346263                 0.042105                    407273              1.000000     493772      23866      58081        1576        251      334033      25045       15966        116075        808378           12.1          0.873684        0.173417           0.957895       0.186995          0.389474         0.197581            0.073684             0.392759                0.978947            0.321626               0.157895         0.3617                         0.5638                        0.9894                           0.9468              4.0            0.668421                36.370                 0.905263              41.26              0.905263  2.407500         0.926316       2.350           0.857895     3.03        0.757895    6.192500           0.526316         5.535             0.536842   
5      bradley     Bradley County, Tennessee          103516        0.873684   314.866397            0.884211                             32.97   4907911.0         0.147368         47.412101                 0.178947                     44532              0.863158       5082       1197       6197         122         30       88814       1654        1954         18634         84882           11.8          0.821053        0.213523           0.894737       0.200471          0.221053         0.146267            0.200000             0.469951                0.663158            0.282631               0.263158         0.3298                         0.3936                        0.8404                           0.8830             36.0            0.894737                25.130                 0.768421              30.39              0.778947  1.995278         0.557895       2.010           0.605263     3.50        0.889474   14.438889           0.157895        12.095             0.231579   
61      monroe      Monroe County, Tennessee           45368        0.652632    71.366703            0.515789                             76.08   1340736.0         0.368421         29.552460                 0.578947                     21413              0.663158        899        187       1966         367          0       41379        248         674          9209         36159           11.1          0.700000        0.280638           0.515789       0.181934          0.484211         0.088895            0.715789             0.514471                0.157895            0.186125               0.831579         0.8298                         0.5745                        0.4362                           0.2979            210.0            0.968421                26.360                 0.800000              33.08              0.821053  1.766667         0.431579       1.740           0.421053     4.20        0.957895   14.005000           0.178947        13.340             0.178947   
65       obion       Obion C

In [10]:
# sort values based on typical risk (pop density only)
tn_df.sort_values('risk_calc_norm', ascending=False).head(10)

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas    gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
49    lawrence    Lawrence County, Tennessee           42479        0.642105    68.833629            0.484211                             75.88   1185737.0         0.400000         27.913487                 0.642105                     18268              0.621053        701        229        892         119         49       39827        137         642          7901         34578           14.5          0.957895        0.294357           0.389474       0.168554          0.652632         0.081711            0.842105             0.507022                0.200000            0.196723               0.800000         0.7766                         0.8404                        0.5319                           0.7021              6.0            0.715789                20.390                 0.600000              23.05              0.489474  2.093333         0.652632       2.175           0.747368     2.31        0.457895    5.700000           0.547368         5.600             0.526316   
78      shelby      Shelby County, Tennessee          924453        1.000000  1210.691352            0.989474                              2.76  65031814.0         0.021053         70.346263                 0.042105                    407273              1.000000     493772      23866      58081        1576        251      334033      25045       15966        116075        808378           12.1          0.873684        0.173417           0.957895       0.186995          0.389474         0.197581            0.073684             0.392759                0.978947            0.321626               0.157895         0.3617                         0.5638                        0.9894                           0.9468              4.0            0.668421                36.370                 0.905263              41.26              0.905263  2.407500         0.926316       2.350           0.857895     3.03        0.757895    6.192500           0.526316         5.535             0.536842   
28    grainger    Grainger County, Tennessee           22870        0.378947    81.507868            0.600000                            100.00    426775.0         0.757895         18.660909                 0.915789                     11214              0.400000         94         75        742          88          0       21622          1         249          5446         17424           10.6          0.621053        0.312462           0.147368       0.172715          0.610526         0.072759            0.905263             0.491739                0.389474            0.218784               0.642105         0.6702                         0.4894                        0.4468                           0.3191             29.0            0.863158                16.685                 0.410526              21.06              0.352632  2.135172         0.694737       2.110           0.715789     4.10        0.942105   11.993571           0.294737        11.360             0.273684   
86       union       Union C

In [11]:
# sort values based on typical risk (both pop info)
tn_df.sort_values('risk_calc_norm_both', ascending=False).head(10)

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas    gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
78      shelby      Shelby County, Tennessee          924453        1.000000  1210.691352            0.989474                              2.76  65031814.0         0.021053         70.346263                 0.042105                    407273              1.000000     493772      23866      58081        1576        251      334033      25045       15966        116075        808378           12.1          0.873684        0.173417           0.957895       0.186995          0.389474         0.197581            0.073684             0.392759                0.978947            0.321626               0.157895         0.3617                         0.5638                        0.9894                           0.9468              4.0            0.668421                36.370                 0.905263              41.26              0.905263  2.407500         0.926316       2.350           0.857895     3.03        0.757895    6.192500           0.526316         5.535             0.536842   
49    lawrence    Lawrence County, Tennessee           42479        0.642105    68.833629            0.484211                             75.88   1185737.0         0.400000         27.913487                 0.642105                     18268              0.621053        701        229        892         119         49       39827        137         642          7901         34578           14.5          0.957895        0.294357           0.389474       0.168554          0.652632         0.081711            0.842105             0.507022                0.200000            0.196723               0.800000         0.7766                         0.8404                        0.5319                           0.7021              6.0            0.715789                20.390                 0.600000              23.05              0.489474  2.093333         0.652632       2.175           0.747368     2.31        0.457895    5.700000           0.547368         5.600             0.526316   
88      warren      Warren County, Tennessee           39922        0.621053    92.268233            0.642105                             61.38   1421164.0         0.357895         35.598517                 0.421053                     18053              0.610526        540        241       3601         141        119       34267        873        1212          8214         31708           12.4          0.905263        0.290366           0.431579       0.148139          0.915789         0.094660            0.642105             0.455339                0.810526            0.218866               0.631579         0.7660                         0.8830                        0.8936                           0.5851              0.0            0.152632                19.190                 0.557895              20.84              0.336842  0.000000         0.157895       0.000           0.157895     0.00        0.157895    0.000000           0.852632         0.000             0.852632   
48  lauderdale  Lauderdale C

#### Places that have had the most earthquakes in TN:  
  
* Lake County | 2118 earthquakes | max mag = 3.3, median mag = 1.5  
* Dyer County | 644 earthquakes | max mag = 4.5, median mag = 1.6  
* Obion County | 391 earthquakes | max mag = 3.3, median mag = 1.5  
* Monroe County | 210 earthquakes | max mag = 4.2, median mag = 1.7  
* Blount County | 135 earthquakes | max mag = 4.7, median mag = 1.9  

Worst case scenario risk scores for these places range from 0.71 (Blount) to 0.97 (Monroe). Typical risk scores range from 0.47 (Blount) to 0.79 (Obion). Either way, the worst case scenario and typical risk scores for these places are arguably pretty high.

In [12]:
# sort values based on number of earthquakes percent rank
tn_df.sort_values('pctrnk_earthquakes', ascending=False).head(10)

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas    gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
47        lake        Lake County, Tennessee            4831        0.010526    29.140159            0.063158                            100.00    210981.0         0.905263         43.672325                 0.210526                      2596              0.010526        981         10        131          11          0        3614         35          94          1114          3717           10.2          0.547368        0.263921           0.663158       0.148209          0.905263         0.089215            0.705263             0.385023                0.989474            0.246076               0.452632         1.0000                         0.5000                        0.4787                           0.9574           2118.0            1.000000                60.610                 1.000000              61.42              1.000000  1.530109         0.336842       1.500           0.326316     3.30        0.836842    6.815432           0.463158         6.770             0.452632   
22        dyer        Dyer County, Tennessee           37057        0.578947    72.329567            0.526316                             42.86   1636436.0         0.326316         44.159970                 0.200000                     16972              0.568421       4762        282       1245         199         53       29748        245         979          7234         29823            9.2          0.294737        0.245784           0.778947       0.189870          0.315789         0.114068            0.378947             0.466304                0.694737            0.256528               0.368421         0.4787                         0.9043                        0.7234                           0.4468            644.0            0.989474                55.630                 0.989474              59.93              0.989474  1.694503         0.400000       1.600           0.352632     4.50        0.984211    7.934517           0.410526         7.700             0.389474   
65       obion       Obion County, Tennessee           30107        0.494737    55.257470            0.378947                             61.58   1126317.0         0.452632         37.410469                 0.378947                     14601              0.505263       2959        113       1271          72         11       25100        546         712          6462         23645           10.5          0.610526        0.307437           0.200000       0.163749          0.705263         0.103830            0.505263             0.460301                0.768421            0.228469               0.568421         0.6277                         0.8830                        0.7872                           0.5000            391.0            0.978947                51.300                 0.978947              57.28              0.978947  1.601151         0.347368       1.510           0.336842     3.30        0.836842    6.849872           0.452632         6.990             0.431579   
61      monroe      Monroe C

In [13]:
# sort values based on probability of experiencing potentially damaging ground-shaking in 100 years
tn_df.sort_values('pctrnk_prob_100y_median', ascending=False).head()

county                   county_full  population_est  pctrnk_pop_est  pop_density  pctrnk_pop_density  pct_people_living_in_rural_areas   gdp_2018  pctrnk_gdp_2018  gdp_per_cap_2018  pctrnk_gdp_per_cap_2018  total_housing_units_2019  pctrnk_housing_units  race_afam  race_asam  race_latn  race_natam  race_paci  race_white  race_othr  race_multi  disability_y  disability_n  pct_uninsured  pctrnk_uninsured  pct_edu_HSgrad  pctrnk_edu_HSgrad  pct_edu_assoc  pctrnk_edu_assoc  pct_edu_min_bac  pctrnk_edu_min_bac  pct_poverty_138_399  pctrnk_poverty_138_399  pct_poverty_400_up  pctrnk_poverty_400_up  socioeconomic  household_comp_and_disability  minority_status_and_language  housing_type_and_transportation  num_earthquakes  pctrnk_earthquakes  pct_prob_100y_median  pctrnk_prob_100y_median  pct_prob_100y_max  pctrnk_prob_100y_max  mag_mean  pctrnk_mag_mean  mag_median  pctrnk_mag_median  mag_max  pctrnk_mag_max  depth_mean  pctrnk_depth_mean  depth_median  pctrnk_depth_median  \
47        lake        Lake County, Tennessee            4831        0.010526    29.140159            0.063158                            100.00   210981.0         0.905263         43.672325                 0.210526                      2596              0.010526        981         10        131          11          0        3614         35          94          1114          3717           10.2          0.547368        0.263921           0.663158       0.148209          0.905263         0.089215            0.705263             0.385023                0.989474            0.246076               0.452632         1.0000                         0.5000                        0.4787                           0.9574           2118.0            1.000000                60.610                 1.000000              61.42              1.000000  1.530109         0.336842       1.500           0.326316     3.30        0.836842    6.815432           0.463158          6.77             0.452632   
22        dyer        Dyer County, Tennessee           37057        0.578947    72.329567            0.526316                             42.86  1636436.0         0.326316         44.159970                 0.200000                     16972              0.568421       4762        282       1245         199         53       29748        245         979          7234         29823            9.2          0.294737        0.245784           0.778947       0.189870          0.315789         0.114068            0.378947             0.466304                0.694737            0.256528               0.368421         0.4787                         0.9043                        0.7234                           0.4468            644.0            0.989474                55.630                 0.989474              59.93              0.989474  1.694503         0.400000       1.600           0.352632     4.50        0.984211    7.934517           0.410526          7.70             0.389474   
65       obion       Obion County, Tennessee           30107        0.494737    55.257470            0.378947                             61.58  1126317.0         0.452632         37.410469                 0.378947                     14601              0.505263       2959        113       1271          72         11       25100        546         712          6462         23645           10.5          0.610526        0.307437           0.200000       0.163749          0.705263         0.103830            0.505263             0.460301                0.768421            0.228469               0.568421         0.6277                         0.8830                        0.7872                           0.5000            391.0            0.978947                51.300                 0.978947              57.28              0.978947  1.601151         0.347368       1.510           0.336842     3.30        0.836842    6.849872           0.452632          6.99             0.431579   
48  lauderdale  Lauderdale Count

#### Counties in TN most likely to experience potentially damaging ground-shaking due to earthquakes:  
  
* Lake County | 2118 earthquakes | max prob = 61.4, median prob = 60.6  
* Dyer County | 644 earthquakes |  max prob = 59.9, median prob = 55.6  
* Obion County | 391 earthquakes |  max prob = 57.3, median prob = 51.3  
* Lauderdale County | 210 earthquakes |  max prob = 54.4, median prob = 48.3  
* Crockett County | 135 earthquakes |  max prob = 47.4, median prob = 43.1  

These may be our counties to watch...